In [1]:
from tree_dataset import TreeDataset
import model as m
from torch.utils.data import DataLoader
import os
import torchvision.transforms as transforms
from d2lvit import *
import copy
import torch.nn as nn

In [2]:
preprocess = transforms.Compose([
    transforms.Grayscale(),
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor()
])
train_set = TreeDataset(os.path.join('..', 'data', 'trainset4'), preprocess) 
val_set = TreeDataset(os.path.join('..', 'data', 'trainset1'), preprocess)
print(f'Train size: {len(train_set)} Val size: {len(val_set)}')
train_loader = DataLoader(train_set, batch_size=32)
val_loader = DataLoader(val_set, batch_size=32)
device = m.get_device()
config = {'labels_key': 'digit_labels'}

Train size: 2000 Val size: 1000
Identified CUDA device: NVIDIA GeForce RTX 3060


In [3]:
model = m.scratch_trained_d2l_vit_digits_model()

In [4]:
train_acc = m.predict(model, train_loader, device, config, None)
print(train_acc)

1.0


In [5]:
val_acc = m.predict(model, val_loader, device, config, None)
print(val_acc)

1.0


In [6]:
digits_model = copy.deepcopy(model)

In [7]:
model.head = nn.Sequential(
    nn.LayerNorm((512,), eps=1e-05, elementwise_affine=True),
    nn.Linear(512, 45)
)
model = model.to(device)

In [8]:
model(torch.unsqueeze(train_set[0]['image'], 0).to(device))

tensor([[ 0.2586, -0.4803,  0.2058,  0.2515, -0.3374,  0.4503, -0.5561, -0.3732,
          0.5386,  0.1772, -0.4163, -0.6740,  0.7919,  0.5417,  0.1346, -1.4765,
         -0.2261,  0.2941, -1.3029, -0.1853,  0.2953, -0.9804, -1.3957, -0.0534,
         -0.0910, -0.1913, -0.2252, -0.5292,  0.3289, -0.3618,  1.1305,  1.6111,
          0.1452,  0.4832, -0.5062, -1.1609,  0.4355, -0.3138, -0.1307,  0.5206,
         -0.2338, -0.2869, -0.2354,  0.5448,  0.9255]], device='cuda:0',
       grad_fn=<AddmmBackward0>)

In [ ]:
m.train(model, 0.0001, 0, 100, train_loader, val_loader, device, os.path.join('..', 'models', 'd2lvit'), digits_model)

Epoch 10 done, train loss: 0.0037 val acc: 0.9010
Epoch 20 done, train loss: 0.0011 val acc: 0.9010
Epoch 30 done, train loss: 0.0059 val acc: 0.9010
Epoch 40 done, train loss: 0.0003 val acc: 0.9010
Epoch 50 done, train loss: 0.0002 val acc: 0.9010
Epoch 60 done, train loss: 0.0001 val acc: 0.9010
Epoch 70 done, train loss: 0.0002 val acc: 0.9010
